In [10]:
from google.colab import drive
drive.mount('/content/drive')
import io
import os
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import math
import itertools

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
archive_path = '/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data'

study_file=os.listdir('/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data')
print(study_file)
test_file = os.listdir('/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/test_data')
print(test_file)

['abortion_counts.csv', 'blm_movement_counts.csv', 'islam_counts.csv', 'espionage_counts.csv', 'human_rights_counts.csv', 'china_counts.csv', 'judaism_counts.csv', 'military_spending_counts.csv', 'guns_counts.csv', 'sexual_harrassment_counts.csv', 'oil_counts.csv', 'russia_counts.csv', 'native_americans_counts.csv', 'palestine_counts.csv', 'tech_censorship_counts.csv', 'universities_counts.csv']
['affirmative_action_counts.csv', 'israel_counts.csv', 'church_state_counts.csv', 'climate_change_counts.csv', 'cuba_counts.csv', 'prisons_counts.csv', 'public_finance_counts.csv', 'nuclear_weapons_counts.csv', 'police__counts.csv', 'private_finance_counts.csv', 'yemen_counts.csv', 'venezuela_counts.csv', 'us_immigration_counts.csv']


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data
for i in range(0, len(study_file)-1):
  globals()['study_data{}'.format(i)] = pd.read_csv(study_file[i], encoding='cp949', header=None)
  globals()['study_data{}'.format(i)] = globals()['study_data{}'.format(i)].drop(1, axis=1)

print(globals()['study_data{}'.format(0)].head)

%cd /content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/test_data
for i in range(0, len(test_file)-1):
  globals()['test_data{}'.format(i)] = pd.read_csv(test_file[i], encoding='cp949', header=None)
  globals()['test_data{}'.format(i)] = globals()['test_data{}'.format(i)].drop(1, axis=1)

/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data
<bound method NDFrame.head of                        0          2         3                     4   \
0                  PHRASE  aljazeera  alternet  americanconservative   
1                pro life         12        13                    68   
2         abortion rights         97        42                    15   
3           anti abortion         53        54                     4   
4             anti choice          0        54                     0   
5           abortion care          7        13                     0   
6               pregnancy         61        69                     8   
7                    kill          2         5                     1   
8        medical abortion          2         0                     0   
9         family planning         18        15                     0   
10           pro abortion          6         1                     3   
11                 murde

In [4]:
for i in range(0, len(study_file)-1):
  a = globals()['study_data{}'.format(i)][0]
  globals()['st_word{}'.format(i)] = a.values.tolist()
  del globals()['st_word{}'.format(i)][0]
#단어 리스트 
print(globals()['st_word{}'.format(0)])

#언론사 리스트
# media_list = globals()['study_data{}'.format(0)].iloc[0].tolist()
# del media_list[0]
for i in range(0, len(study_file)-1):
  globals()['media_list{}'.format(i)] = globals()['study_data{}'.format(i)].iloc[0].tolist()
  del globals()['media_list{}'.format(i)][0]
print(media_list0)
print(len(media_list0))
#리스트의 0 값은 PHRASE이므로 삭제해줌.

['pro life', 'abortion rights', 'anti abortion', 'anti choice', 'abortion care', 'pregnancy', 'kill', 'medical abortion', 'family planning', 'pro abortion', 'murder', 'fetus', 'mother', 'reproductive rights', 'infanticide', 'abortion access', 'abortion ban', 'infant', 'reproductive health', 'pro choice', 'abortion law', 'medication abortion', 'contraception', 'fetal heartbeat', 'fetal remains', 'abortion services', 'born alive', 'birth control', 'religious liberty', 'aborted', 'abortion rules', 'abortion providers', 'reproductive justice', 'womens rights', 'dead', 'termination', 'abortion clinic', 'adoption', 'rape or incest', 'maternal', 'abortion restrictions', 'abortion pills', 'unborn babies', 'abortionists', 'fetal abnormalities', 'human life', 'heartbeat bill', 'access to abortion', 'abortion industry', 'right to life']
['aljazeera', 'alternet', 'americanconservative', 'americanspectator', 'ap', 'atlantic', 'bbc', 'breitbart', 'buzzfeed', 'cbs', 'cnn', 'commondreams', 'conversati

In [5]:
#최대 단어 3개 뽑을때 사용할 csv 전처리(object 자료형 변환을 위해 삭제)
for i in range(0, len(study_file)-1):
  globals()['st_d{}'.format(i)]=globals()['study_data{}'.format(i)].drop(0).drop(0,axis=1)
  globals()['st_d{}'.format(i)]=globals()['st_d{}'.format(i)].apply(pd.to_numeric)
  # globals()['st_d{}'.format(i)]=globals()['st_d{}'.format(i)].to_numpy()
  globals()['st_d{}'.format(i)].columns = globals()['study_data{}'.format(i)].iloc[0,1:]
  globals()['st_d{}'.format(i)].reset_index(drop=True)
  globals()['st_d{}'.format(i)]=globals()['st_d{}'.format(i)].set_index(globals()['study_data{}'.format(i)].iloc[1:,0])
  # globals()['stu_d{}'.format(i)] = pd.DataFrame(globals()['st_d{}'.format(i)], columns=globals()['study_data{}'.format(i)].iloc[0,1:], index=globals()['study_data{}'.format(i)].iloc[1:,0])

# globals()['st_d0'][42]=globals()['st_d0'][43]
print(len(globals()['st_d{}'.format(0)]))
# print(len(globals()['st_d{}'.format(4)]))
globals()['st_d{}'.format(0)]


50


,aljazeera,alternet,americanconservative,americanspectator,ap,atlantic,bbc,breitbart,buzzfeed,cbs,...,rt,slate,spectator,townhall,truthdig,usatoday,vice,vox,wapo,wsj
0,,,,,,,,,,,,,,,,,,,,,
pro life,12,13,68,10,7,16,0,200,48,63,...,22,59,52,26,5,131,13,60,242,1
abortion rights,97,42,15,5,43,63,10,54,151,282,...,3,71,4,0,32,211,103,461,585,24
anti abortion,53,54,4,0,37,49,17,38,188,208,...,11,107,5,0,27,239,143,487,29,3
anti choice,0,54,0,0,0,1,0,8,4,7,...,1,10,0,1,5,5,38,14,19,0
abortion care,7,13,0,1,2,2,1,25,33,40,...,0,27,1,0,7,52,60,76,64,1
pregnancy,61,69,8,3,26,93,17,113,172,306,...,14,112,34,5,29,325,262,487,531,26
kill,2,5,1,1,0,3,0,13,3,2,...,17,8,19,1,1,21,6,29,14,0
medical abortion,2,0,0,0,1,0,4,4,21,2,...,0,2,0,0,0,3,2,8,9,0
family planning,18,15,0,1,43,10,3,50,42,66,...,0,8,3,0,27,39,12,72,130,9


In [6]:
def kth_largest_number(arr, K):
  unique_nums = list(arr)
  sorted_nums = sorted(unique_nums, reverse=True)
  return sorted_nums[K-1]

In [7]:
#가장 숫자 높은 단어 세개씩만 각 열마다...
for k in range(0, len(study_file)-1):
  globals()['word_3{}'.format(k)]=[]
  for i in range(0, len(globals()['media_list{}'.format(k)])-1):
    # data.append(kth_largest_number(globals()['st_d{}'.format(k)][i], 1))
    # print(kth_largest_number(globals()['st_d{}'.format(k)][i], 1))
    df = globals()['st_d{}'.format(k)]
    a = df.nlargest(3, globals()['media_list{}'.format(k)][i])
    A = a.index.to_list()
    # print(A)
    globals()['word_3{}'.format(k)].append(A)
    # data.append(df.sort_values(by=media_list[i-2], ascending=False).groupby(globals()['st_word{}'.format(i-2)]).head(3))
  # globals()['st_top{}'.format(k)] = pd.DataFrame(data, columns=['x','y','z'], index=globals()['media_list{}'.format(k)])

# print(globals()['st_top{}'.format(0)])
print(len(globals()['word_3{}'.format(0)]))

globals()['word_3{}'.format(0)]

45


[['abortion rights', 'pregnancy', 'anti abortion'],
 ['pregnancy', 'birth control', 'anti abortion'],
 ['pro life', 'abortion rights', 'mother'],
 ['pro life', 'dead', 'contraception'],
 ['abortion rights', 'family planning', 'anti abortion'],
 ['pregnancy', 'abortion rights', 'anti abortion'],
 ['anti abortion', 'pregnancy', 'fetus'],
 ['pro life', 'pregnancy', 'abortion rights'],
 ['anti abortion', 'pregnancy', 'abortion rights'],
 ['pregnancy', 'abortion rights', 'anti abortion'],
 ['pregnancy', 'abortion rights', 'anti abortion'],
 ['abortion care', 'pregnancy', 'anti choice'],
 ['pregnancy', 'medical abortion', 'reproductive health'],
 ['anti abortion', 'fetus', 'pregnancy'],
 ['pro life', 'pregnancy', 'mother'],
 ['pregnancy', 'anti abortion', 'abortion rights'],
 ['pregnancy', 'anti abortion', 'abortion rights'],
 ['pro life', 'pregnancy', 'aborted'],
 ['pregnancy', 'abortion rights', 'pro choice'],
 ['pro life', 'pregnancy', 'mother'],
 ['pro life', 'pregnancy', 'mother'],
 ['p

In [13]:
for k in range(0, len(study_file)-1):
  data = globals()['word_3{}'.format(k)]
  mapping = {item: idx for idx, item in enumerate(globals()['st_word{}'.format(k)])}
  print(mapping)

  mapped_data = [[mapping[val] for val in lst] for lst in data]

  print(mapped_data)

  data_int = []
  for point in data:
      point_int = [mapping[label] for label in point]
      data_int.append(point_int)

  # 축 레이블 리스트
  axis_labels = list(mapping.keys())

  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(111, projection='3d')

  # 축 범위 지정
  ax.set_xlim(0, len(mapping) - 1)
  ax.set_ylim(0, len(mapping) - 1)
  ax.set_zlim(0, len(mapping) - 1)


  # 축 레이블 지정
  ax.set_xticks(range(len(axis_labels)))
  ax.set_yticks(range(len(axis_labels)))
  ax.set_zticks(range(len(axis_labels)))
  ax.set_xticklabels(axis_labels, fontsize=4, rotation=90)
  ax.set_yticklabels(axis_labels, fontsize=4, rotation=0)
  ax.set_zticklabels(axis_labels, fontsize=4, rotation=0)

  # 데이터 플롯
  x = [point[0] for point in data_int]
  y = [point[1] for point in data_int]
  z = [point[2] for point in data_int]
  ax.scatter(x, y, z)

  labels = globals()['media_list{}'.format(k)]

  for i, label in enumerate(labels[:len(data_int)]):
      ax.text(x[i], y[i], z[i], label, fontsize=6)
  filename=f'/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/study_image/image_{k}.png'
  plt.savefig(filename, dpi=300)
  # plt.show()
  

Output hidden; open in https://colab.research.google.com to view.

In [36]:
%cd /content/drive/MyDrive/Colab Notebooks/3113_data_analyze/study_distance/
for k in range(0, len(study_file)-1):

  data = globals()['word_3{}'.format(k)]
  mapping = {item: idx for idx, item in enumerate(globals()['st_word{}'.format(k)])}
  media_list = globals()['media_list{}'.format(k)]
  print(mapping)

  # 새로운 데이터프레임 행, 열 인덱스 지정
  columns = media_list
  index = media_list

  mapped_data = [[mapping[val] for val in lst] for lst in data]

  data_int = []
  for point in data:
      point_int = [mapping[label] for label in point]
      data_int.append(point_int)

  distances = []
  for i, pair in enumerate(itertools.combinations(data_int, 2)):
      x1, y1, z1 = pair[0]
      x2, y2, z2 = pair[1]
      distance = ((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)**0.5
      distances.append(distance)

  df = pd.DataFrame(columns=columns, index=index)
  for i in range(len(media_list)):
      for j in range(len(media_list)):
          if i == j:
              df.iloc[i, j] = np.nan
          elif i > j:
              idx = 3*(j-1) - (j-1)*(j-2)//2 + (i-j-1)
              df.iloc[i, j] = distances[idx]
          else:
              idx = 3*(i-1) - (i-1)*(i-2)//2 + (j-i-1)
              df.iloc[i, j] = distances[idx]

  # display(df)
  file_name = study_file[k] + 'distance.csv'
  df.to_csv(file_name, index=True)
  print(f"{file_name} 파일이 저장되었습니다.")


/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/study_distance
{'pro life': 0, 'abortion rights': 1, 'anti abortion': 2, 'anti choice': 3, 'abortion care': 4, 'pregnancy': 5, 'kill': 6, 'medical abortion': 7, 'family planning': 8, 'pro abortion': 9, 'murder': 10, 'fetus': 11, 'mother': 12, 'reproductive rights': 13, 'infanticide': 14, 'abortion access': 15, 'abortion ban': 16, 'infant': 17, 'reproductive health': 18, 'pro choice': 19, 'abortion law': 20, 'medication abortion': 21, 'contraception': 22, 'fetal heartbeat': 23, 'fetal remains': 24, 'abortion services': 25, 'born alive': 26, 'birth control': 27, 'religious liberty': 28, 'aborted': 29, 'abortion rules': 30, 'abortion providers': 31, 'reproductive justice': 32, 'womens rights': 33, 'dead': 34, 'termination': 35, 'abortion clinic': 36, 'adoption': 37, 'rape or incest': 38, 'maternal': 39, 'abortion restrictions': 40, 'abortion pills': 41, 'unborn babies': 42, 'abortionists': 43, 'fetal abnormalities': 44, 'human life'

,aljazeera,alternet,americanconservative,americanspectator,ap,atlantic,bbc,breitbart,buzzfeed,cbs,...,rt,slate,spectator,townhall,truthdig,usatoday,vice,vox,wapo,wsj
aljazeera,NaN,5.09902,4.242641,5.09902,1.414214,22.36068,10.816654,35.24202,3.0,5.656854,...,18.867962,10.049876,1.414214,7.681146,3.316625,5.09902,9.539392,26.324893,0.0,5.09902
alternet,5.09902,NaN,22.36068,10.816654,35.24202,3.0,5.656854,9.055385,1.414214,1.414214,...,7.681146,3.316625,5.09902,9.539392,26.324893,0.0,5.09902,5.09902,1.414214,2.0
americanconservative,4.242641,22.36068,NaN,3.0,5.656854,9.055385,1.414214,1.414214,5.656854,5.656854,...,5.09902,9.539392,26.324893,0.0,5.09902,5.09902,1.414214,2.0,28.301943,21.771541
americanspectator,5.09902,10.816654,3.0,NaN,9.055385,1.414214,1.414214,5.656854,5.656854,3.162278,...,9.539392,26.324893,0.0,5.09902,5.09902,1.414214,2.0,28.301943,21.771541,19.416488
ap,1.414214,35.24202,5.656854,9.055385,NaN,1.414214,1.414214,5.656854,5.656854,3.162278,...,9.539392,26.324893,0.0,5.09902,5.09902,1.414214,2.0,28.301943,21.771541,19.416488
atlantic,22.36068,3.0,9.055385,1.414214,1.414214,NaN,1.414214,1.414214,5.656854,5.656854,...,5.09902,9.539392,26.324893,0.0,5.09902,5.09902,1.414214,2.0,28.301943,21.771541
bbc,10.816654,5.656854,1.414214,1.414214,1.414214,1.414214,NaN,9.055385,1.414214,1.414214,...,7.681146,3.316625,5.09902,9.539392,26.324893,0.0,5.09902,5.09902,1.414214,2.0
breitbart,35.24202,9.055385,1.414214,5.656854,5.656854,1.414214,9.055385,NaN,3.0,5.656854,...,18.867962,10.049876,1.414214,7.681146,3.316625,5.09902,9.539392,26.324893,0.0,5.09902
buzzfeed,3.0,1.414214,5.656854,5.656854,5.656854,5.656854,1.414214,3.0,NaN,22.36068,...,5.09902,0.0,6.0,5.656854,18.867962,10.049876,1.414214,7.681146,3.316625,5.09902
cbs,5.656854,1.414214,5.656854,3.162278,3.162278,5.656854,1.414214,5.656854,22.36068,NaN,...,11.224972,29.017236,11.401754,5.09902,1.414214,5.09902,0.0,6.0,5.656854,18.867962


abortion_counts.csvdistance.csv 파일이 저장되었습니다.
{'Black Lives Matter': 0, 'defund the police': 1, 'demonstrations': 2, 'demonstrators': 3, 'mob': 4, 'protest': 5, 'racial injustice': 6, 'rally': 7, 'rioters': 8, 'riots': 9, 'anarchists': 10, 'protesters': 11, 'police brutality': 12}


,aljazeera,alternet,americanconservative,americanspectator,ap,atlantic,axios,bbc,breitbart,businessinsider,...,time,townhall,truthdig,usatoday,verge,veteranstoday,vice,vox,wapo,wsj
aljazeera,NaN,6.403124,9.899495,9.0,7.28011,0.0,9.433981,11.445523,9.0,0.0,...,10.049876,0.0,12.0,5.656854,0.0,11.74734,12.0,0.0,5.0,7.071068
alternet,6.403124,NaN,0.0,9.433981,11.445523,9.0,0.0,5.0,13.892444,8.602325,...,5.656854,0.0,11.74734,12.0,0.0,5.0,7.071068,0.0,9.899495,9.0
americanconservative,9.899495,0.0,NaN,9.0,0.0,5.0,13.892444,8.602325,9.899495,0.0,...,11.74734,12.0,0.0,5.0,7.071068,0.0,9.899495,9.0,9.433981,11.445523
americanspectator,9.0,9.433981,9.0,NaN,5.0,13.892444,8.602325,9.899495,0.0,12.0,...,12.0,0.0,5.0,7.071068,0.0,9.899495,9.0,9.433981,11.445523,9.0
ap,7.28011,11.445523,0.0,5.0,NaN,13.892444,8.602325,9.899495,0.0,12.0,...,12.0,0.0,5.0,7.071068,0.0,9.899495,9.0,9.433981,11.445523,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
veteranstoday,11.74734,5.0,0.0,9.899495,9.899495,0.0,5.0,11.74734,0.0,9.899495,...,2.0,9.899495,0.0,3.0,9.899495,NaN,5.385165,8.602325,11.045361,6.403124
vice,12.0,7.071068,9.899495,9.0,9.0,9.899495,7.071068,12.0,12.0,4.0,...,3.605551,5.656854,9.486833,0.0,1.414214,5.385165,NaN,9.899495,7.615773,13.892444
vox,0.0,0.0,9.0,9.433981,9.433981,9.0,0.0,0.0,5.656854,6.0,...,12.165525,12.0,8.602325,12.0,8.062258,8.602325,9.899495,NaN,12.369317,7.071068
wapo,5.0,9.899495,9.433981,11.445523,11.445523,9.433981,9.899495,5.0,0.0,9.0,...,3.605551,9.0,8.944272,4.242641,9.899495,11.045361,7.615773,12.369317,NaN,0.0


blm_movement_counts.csvdistance.csv 파일이 저장되었습니다.
{'anti Muslim': 0, 'bigotry': 1, 'blasphemy': 2, 'caliph': 3, 'citizenship law': 4, 'discrimination': 5, 'extremism': 6, 'extremist': 7, 'free speech': 8, 'freedom of expression': 9, 'hate crimes': 10, 'hate groups': 11, 'holy month': 12, 'ideology': 13, 'Islamophobia in the': 14, 'migrant': 15, 'militants': 16, 'mosque': 17, 'Muslim ban': 18, 'nationalism': 19, 'nationalist': 20, 'non Muslim': 21, 'peace': 22, 'prayer': 23, 'refugees': 24, 'religious freedom': 25, 'religious minorities': 26, 'religious violence': 27, 'terror attacks': 28, 'terrorism': 29, 'terrorist': 30, 'travel ban': 31, 'worship': 32, 'worshippers': 33, 'beheaded': 34, 'genocide': 35, 'jihad': 36, 'pilgrimage': 37}


...

native_americans_counts.csvdistance.csv 파일이 저장되었습니다.
{'annexation': 0, 'anti Israel': 1, 'anti Palestinian': 2, 'Apartheid Israel': 3, 'Arab countries': 4, 'Arab League': 5, 'BDS movement': 6, 'boycott': 7, 'coalition': 8, 'diplomatic': 9, 'factions': 10, 'fascism': 11, 'Israel lobby': 12, 'Jewish state': 13, 'Joint List': 14, 'occupation': 15, 'Palestinian state': 16, 'peace plan': 17, 'pro Israel': 18, 'sanctions': 19, 'siege': 20, 'solidarity': 21, 'sovereignty': 22, 'State of Israel': 23, 'statehood': 24, 'terrorism': 25, 'the PFLP': 26, 'two state solution': 27, 'Palestinian Authority': 28, 'Palestine Liberation Organization': 29, 'colonial': 30}


,aljazeera,alternet,americanconservative,americanspectator,americanthinker,antiwar,ap,atlantic,bbc,breitbart,...,slate,socialistproject,spectator,strategicculture,truthdig,usatoday,veteranstoday,vice,vox,wapo
aljazeera,NaN,12.247449,13.638182,18.138357,8.306624,37.536649,20.639767,16.093477,39.711459,23.043437,...,28.861739,27.658633,19.235384,16.186414,36.262929,19.697716,32.07803,27.477263,15.0,19.924859
alternet,12.247449,NaN,37.536649,20.639767,16.093477,39.711459,23.043437,13.076697,20.124612,20.808652,...,16.186414,36.262929,19.697716,32.07803,27.477263,15.0,19.924859,19.235384,21.679483,26.134269
americanconservative,13.638182,37.536649,NaN,39.711459,23.043437,13.076697,20.124612,20.808652,14.0,7.0,...,19.697716,32.07803,27.477263,15.0,19.924859,19.235384,21.679483,26.134269,28.79236,32.403703
americanspectator,18.138357,20.639767,39.711459,NaN,13.076697,20.124612,20.808652,14.0,7.0,35.0,...,32.07803,27.477263,15.0,19.924859,19.235384,21.679483,26.134269,28.79236,32.403703,8.602325
americanthinker,8.306624,16.093477,23.043437,13.076697,NaN,20.124612,20.808652,14.0,7.0,35.0,...,32.07803,27.477263,15.0,19.924859,19.235384,21.679483,26.134269,28.79236,32.403703,8.602325
antiwar,37.536649,39.711459,13.076697,20.124612,20.124612,NaN,20.124612,20.808652,14.0,7.0,...,19.697716,32.07803,27.477263,15.0,19.924859,19.235384,21.679483,26.134269,28.79236,32.403703
ap,20.639767,23.043437,20.124612,20.808652,20.808652,20.124612,NaN,13.076697,20.124612,20.808652,...,16.186414,36.262929,19.697716,32.07803,27.477263,15.0,19.924859,19.235384,21.679483,26.134269
atlantic,16.093477,13.076697,20.808652,14.0,14.0,20.808652,13.076697,NaN,39.711459,23.043437,...,28.861739,27.658633,19.235384,16.186414,36.262929,19.697716,32.07803,27.477263,15.0,19.924859
bbc,39.711459,20.124612,14.0,7.0,7.0,14.0,20.124612,39.711459,NaN,37.536649,...,21.954498,27.513633,20.124612,36.124784,28.861739,27.658633,19.235384,16.186414,36.262929,19.697716
breitbart,23.043437,20.808652,7.0,35.0,35.0,7.0,20.808652,23.043437,37.536649,NaN,...,26.70206,22.737634,15.811388,14.3527,13.190906,21.954498,27.513633,20.124612,36.124784,28.861739


palestine_counts.csvdistance.csv 파일이 저장되었습니다.
{'Big Tech': 0, 'conservative': 1, 'big tech': 2, 'fake news': 3, 'tech companies': 4, 'online platforms': 5, 'misinformation': 6, 'hate speech': 7, 'disinformation': 8, 'news organizations': 9, 'independent media': 10, 'Internet freedom': 11, 'bias': 12, 'purge': 13, 'social media giants': 14, 'to censor': 15, 'liability': 16, 'censored content': 17, 'triggered': 18, 'Fake News': 19, 'alternative media': 20, 'social media site': 21, 'media networks': 22, 'online censorship': 23, 'content moderation': 24, 'moderators': 25, 'leftist': 26, 'propaganda': 27, 'to control': 28, 'social media companies': 29, 'inauthentic behavior': 30, 'mainstream': 31, 'social media platforms': 32}


,aljazeera,alternet,americanconservative,ap,atlantic,bbc,breitbart,businessinsider,buzzfeed,cbs,...,slate,spectator,techcrunch,time,usatoday,verge,vice,vox,wapo,wsj
aljazeera,NaN,11.0,19.0,8.602325,26.400758,36.11094,11.0,10.29563,15.524175,40.459857,...,9.0,8.485281,7.28011,12.369317,13.96424,30.675723,4.123106,7.615773,26.0,11.045361
alternet,11.0,NaN,36.11094,11.0,10.29563,15.524175,40.459857,14.696938,24.083189,28.071338,...,12.369317,13.96424,30.675723,4.123106,7.615773,26.0,11.045361,9.433981,27.349589,1.0
americanconservative,19.0,36.11094,NaN,15.524175,40.459857,14.696938,24.083189,28.071338,26.07681,3.605551,...,30.675723,4.123106,7.615773,26.0,11.045361,9.433981,27.349589,1.0,32.695565,29.086079
ap,8.602325,11.0,15.524175,NaN,14.696938,24.083189,28.071338,26.07681,3.605551,5.830952,...,4.123106,7.615773,26.0,11.045361,9.433981,27.349589,1.0,32.695565,29.086079,31.76476
atlantic,26.400758,10.29563,40.459857,14.696938,NaN,24.083189,28.071338,26.07681,3.605551,5.830952,...,4.123106,7.615773,26.0,11.045361,9.433981,27.349589,1.0,32.695565,29.086079,31.76476
bbc,36.11094,15.524175,14.696938,24.083189,24.083189,NaN,24.083189,28.071338,26.07681,3.605551,...,30.675723,4.123106,7.615773,26.0,11.045361,9.433981,27.349589,1.0,32.695565,29.086079
breitbart,11.0,40.459857,24.083189,28.071338,28.071338,24.083189,NaN,14.696938,24.083189,28.071338,...,12.369317,13.96424,30.675723,4.123106,7.615773,26.0,11.045361,9.433981,27.349589,1.0
businessinsider,10.29563,14.696938,28.071338,26.07681,26.07681,28.071338,14.696938,NaN,15.524175,40.459857,...,9.0,8.485281,7.28011,12.369317,13.96424,30.675723,4.123106,7.615773,26.0,11.045361
buzzfeed,15.524175,24.083189,26.07681,3.605551,3.605551,26.07681,24.083189,15.524175,NaN,36.11094,...,7.0,3.605551,28.160256,32.449961,9.0,8.485281,7.28011,12.369317,13.96424,30.675723
cbs,40.459857,28.071338,3.605551,5.830952,5.830952,3.605551,28.071338,40.459857,36.11094,NaN,...,3.0,24.083189,17.0,14.035669,12.369317,7.0,3.605551,28.160256,32.449961,9.0


tech_censorship_counts.csvdistance.csv 파일이 저장되었습니다.
